In [2]:
import os
import requests
from bs4 import BeautifulSoup as BS
from pprint import pprint

def check_tag(tag, subtag, check):
    try:
        if check in tag[subtag]:
            return True
        return False
    except KeyError:
        return False
    
pos_trans = {'ILB': ['ILB','LB'],
            'OLB': ['OLB','LB'],
            'FS': ['S','DB'],
            'SS': ['S','DB'],
            'OT': ['OT', 'OL', 'T'],
            'OG': ['OG', 'OL','G'],
            'C': ['C', 'OL'],
            'QB': ['QB'],
            'RB': ['RB'],
            'WR': ['WR'],
            'TE': ['TE'],
            'CB': ['CB', 'DB'],
            'DE': ['DE'],
            'DT': ['DT'],
            'FB': ['FB']}

In [16]:
CDo = open('COMBINE_DATA.csv')
full = [cd.split(',') for cd in CDo.read().split('\n')]
headers = full[0]
CD = full[1:-1]

base_url = "http://www.pro-football-reference.com/search/search.fcgi?hint=%s+%s&search=%s+%s&pid="
pf_url = "http://www.pro-football-reference.com"
url_dict = {}

final_str = ''
for header in headers:
    final_str += header + ','
final_str += 'G,GS,G3,GS3\n'

for player in range(326, len(CD)):
    if player % 100 == 0:
        print('\n%s / 3124\n' % str(player))
    name = CD[player][1].replace('\\','')
    CD[player][1] = name
    ns = name.split(' ')
    if len(ns) != 2:
        ns = [ns[0], '%s+%s' % (ns[1],ns[2])]
        
    player_url = base_url % (ns[0], ns[1], ns[0], ns[1])
    response = requests.get(player_url)
    html = response.text.encode('utf-8').__str__()
    n = html.find("<html")
    player_soup = BS(html[n:], "html.parser")
    title_list = player_soup.find_all('title')
    
    player_page = 1
    if 'Search Results' in title_list[0].contents[0]:
        player_page = 0
        
    if not player_page:
        div_list = player_soup.find_all('div')
        for div in div_list:
            if check_tag(div, 'id', 'players'):
                div_list2 = div.find_all('div')
                if div_list2:
                    for div2 in div_list2:
                        try:
                            pos = div2.div.contents[2].split(' ')[2]
                            year = div2.div.contents[2].split(' ')[3][1:5]
                            if pos in pos_trans[CD[player][3]]:
                                if year == CD[player][0]:
                                    player_url = pf_url + div2.div.a['href']
                                    response = requests.get(player_url)
                                    html = response.text.encode('utf-8').__str__()
                                    n = html.find("<html")
                                    player_soup = BS(html[n:], "html.parser")
                                    title_list = player_soup.find_all('title')
                                    break
                        except AttributeError:
                            pass
                    break
    
    resl = [0,0,0,0,0]
    
    if 'Search Results' not in title_list[0].contents[0]:
        table_list = player_soup.find_all('table')
        for table in table_list:
            if check_tag(table, 'class', 'row_summable') and check_tag(table, 'class', 'sortable') and not check_tag(table, 'id', 'passing_clone'):
                th_list = table.thead.find_all('th')
                year = False
                for th in th_list:
                    if check_tag(th, 'aria-label', 'Year'):
                        year = True
                        break
                if year:
                    tr_list = table.tbody.find_all('tr')
                    for tr in tr_list:
                        td_list = tr.find_all('td')
                        if len(tr.th.contents) > 0:
                            try:
                                if tr.th.a.contents[0] == '2016':
                                    resl[4] += (11/16)
                                else:
                                    resl[4] += 1
                            except AttributeError:
                                if tr.th.contents[0] == '2016':
                                    resl[4] += (11/16)
                                else:
                                    resl[4] += 1
                            for td in td_list:
                                if check_tag(td, 'data-stat', 'gs'):
                                    if len(td.contents) > 0:
                                        gs = int(td.contents[0])
                                        resl[1] += gs
                                        if resl[4] > 3:
                                            resl[3] += gs
                                elif td['data-stat'] == 'g':
                                    if len(td.contents) > 0:
                                        g = int(td.contents[0])
                                        resl[0] += g
                                        if resl[4] > 3:
                                            resl[2] += g
    
    res0, res1, res2, res3 = 0, 0, 0, 0
    if resl[4] > 0:
        res0 = resl[0]/resl[4]
        res1 = resl[1]/resl[4]
        if resl[4] > 3:
            res2 = resl[2]/(resl[4] - 3)
            res3 = resl[3]/(resl[4] - 3)
    print(name, res0, res1, res2, res3, resl[4])
    CD[player].append(str(res0))
    CD[player].append(str(res1))
    CD[player].append(str(res2))
    CD[player].append(str(res3))
    for p in CD[player]:
        final_str += p + ','
    final_str += '\n'

print('DONE')

Andrew Luck 13.866666666666667 13.866666666666667 10.074074074074074 10.074074074074074 4.6875
Terrell Manning 0 0 0 0 0
Brandon Marshall 9.6 7.8933333333333335 14.814814814814815 14.222222222222221 4.6875
Doug Martin 11.306666666666667 11.306666666666667 11.851851851851851 11.851851851851851 4.6875
Keshawn Martin 12.373333333333333 2.1333333333333333 5.925925925925926 4.7407407407407405 4.6875
Mike Martin 11.5 1.25 5.0 0.0 4
Jonathan Massaquoi 13.0 2.3333333333333335 0 0 3
Bobby Massie 13.44 11.733333333333333 13.62962962962963 13.62962962962963 4.6875
Rishard Matthews 12.8 4.906666666666666 13.037037037037036 10.074074074074074 4.6875
Antoine McClain 0 0 0 0 0
Shea McClellin 12.8 6.826666666666667 11.851851851851851 7.111111111111111 4.6875
Kelcie McCray 12.16 1.4933333333333334 15.407407407407407 4.148148148148148 4.6875
Jerron McMillian 14.0 1.0 0 0 2
Marvin McNutt 0 0 0 0 0
Caleb McSurdy 0 0 0 0 0
Vaughn Meatoga 0 0 0 0 0
Davin Meggett 0 0 0 0 0
Whitney Mercilus 15.573333333333334

KeyboardInterrupt: 

In [17]:
fs = open('OUTPUT.csv').read()

o = open('OUTPUT.csv', 'w')
o.write(fs + final_str)
o.close()

In [ ]:
combine_df=pd.DataFrame.from_csv